In [4]:
#https://google.github.io/mediapipe/

import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
import numpy as np

src = cv2.imread("Cropped Image.jpg")
# Define an array of endpoints of triangle
points = np.array([[14, 14],[160, 1],[158, 17],[126, 45],[95, 52],[54, 52],[22 ,28]], np.int32)

foreground = np.ones((100,100,3),dtype='uint8')*255
# Set initial value of weights
alpha = 0.4

# For static images:
IMAGE_FILES = []
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    # Convert the BGR image to RGB before processing.
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print and draw face mesh landmarks on the image.
    if not results.multi_face_landmarks:
      continue
    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
      print('face_landmarks:', face_landmarks)
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_TESSELATION,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_tesselation_style())
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_CONTOURS,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_contours_style())
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_iris_connections_style())
    cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)
    
# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

# https://www.youtube.com/watch?v=Yg6bFRnOSbs

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        
        src_mask = np.zeros(src.shape, src.dtype)

        #image = cv2.imread('sorriso.jpeg')
        rgb_image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        source = face_landmarks.landmark[13]
        target = face_landmarks.landmark[14]
        relative_source = (int(source.x * image.shape[1]), int(source.y * image.shape[0]))
        relative_target = (int(target.x * image.shape[1]), int(target.y * image.shape[0]))
        center = (int((relative_source[0] + relative_target[0])/2),int((relative_source[1] + relative_target[1])/2)-5)
    
    cv2.fillPoly(src_mask, [points], (255, 255, 255))

    scale_percent = 60 # percent of original size
    width = int(src.shape[1] * scale_percent / 100)
    height = int(src.shape[0] * scale_percent / 100)
    dim = (width, height)
      
    # resize image
    resized = cv2.resize(src, dim, interpolation = cv2.INTER_AREA)
    image = cv2.seamlessClone(src, image, src_mask, center, cv2.NORMAL_CLONE)

    print("background position {}".format(center))
    #background = image
    #background = cv2.flip(image, 1)
    #added_image = cv2.addWeighted(background[center[1]-50:center[1]+50,center[0]-50:center[0]+50,:],alpha,foreground[0:100,0:100,:],1-alpha,0)
    #background[center[1]-50:center[1]+50,center[0]-50:center[0]+50] = added_image
    #cv2.imshow('MediaPipe Face Mesh', cv2.flip(background, 1))
    #background = cv2.flip(background, 1)
    cv2.imshow('MediaPipe Face Mesh', image)
    #cv2.imshow('MediaPipe Face Mesh', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.5) /io/opencv/modules/core/src/matrix.cpp:810: error: (-215:Assertion failed) 0 <= roi.x && 0 <= roi.width && roi.x + roi.width <= m.cols && 0 <= roi.y && 0 <= roi.height && roi.y + roi.height <= m.rows in function 'Mat'


In [15]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
#mp_drawing_styles = mp.solutions.drawing_styles
#mp_face_mesh = mp.solutions.face_mesh
# https://www.youtube.com/watch?v=mCcPmlr7y3U&t=2s

webcam = cv2.VideoCapture(0)
sol_rec_face = mp.solutions.face_detection
rec_face = sol_rec_face.FaceDetection()
desenho = mp.solutions.drawing_utils

print("start tutorial")

while True:
    verify, frame =  webcam.read()

    if not verify:
        break

    face_list = rec_face.process(frame)

    if face_list.detections:
        for face in face_list.detections:
            desenho.draw_detection(frame, face)

    cv2.imshow("faces", frame)

    # quando aperta ESC finaliza
    if cv2.waitKey(5) == 27:
        break

webcam.release()
cv2.destroyAllWindows()

start tutorial


In [16]:
from unittest import skip
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
#mp_drawing = mp.solutions.drawing_utils
#mp_drawing_styles = mp.solutions.drawing_styles
#mp_face_mesh = mp.solutions.face_mesh
# https://www.youtube.com/watch?v=mCcPmlr7y3U&t=2s

#mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

image = cv2.imread('sorriso.jpeg')
rgb_image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

height, width, _ = image.shape

results = face_mesh.process(rgb_image)

#type(results.multi_face_landmarks)
#results.multi_face_landmarks

#mp_face_mesh.FACEMESH_LIPS

facial_landmarks = results.multi_face_landmarks[0]
#for facial_landmarks in results.multi_face_landmarks:
#    pt1 = facial_landmarks.landmark[0]
#    x = int(pt1.x * width)
#    y = int(pt1.y * height)
#    cv2.circle(image, (x,y), 7 ,(100,100,0), -1)


#print(type(mp_face_mesh.FACEMESH_LIPS))
print(mp_face_mesh.FACEMESH_LIPS)
count = 0
for source_idx, target_idx in mp_face_mesh.FACEMESH_LIPS:
    
    #if count > 20:
    #    print('enter')
    #else:
    source = facial_landmarks.landmark[source_idx]
    target = facial_landmarks.landmark[target_idx]
    relative_source = (int(source.x * image.shape[1]), int(source.y * image.shape[0]))
    relative_target = (int(target.x * image.shape[1]), int(target.y * image.shape[0]))
    #if source_idx < 150 and target_idx < 100:
    if source_idx == 13:
        
        print(source)

        
        #cv2.line(image, relative_source,relative_target, color= (255,255,255), thickness= 2)
        print(relative_source)
        print(relative_target)
        cv2.circle(image, relative_source, radius=4, color=(0, 0, 255), thickness=-1)
        #cv2.circle(image, relative_target, radius=4, color=(0, 255, 0), thickness=-1)
        
        # font
        font = cv2.FONT_HERSHEY_COMPLEX_SMALL
        
        # org
        org = (50, 50)
        
        # fontScale
        fontScale = .5
        
        # Blue color in BGR
        color = (255, 0, 0)
        
        # Line thickness of 2 px
        thickness = 2
        
        # Using cv2.putText() method
        image = cv2.putText(image, str(source_idx), relative_source, font, 
                        fontScale, color, thickness, cv2.LINE_AA)

    #cv2.line(image, relative_source,relative_target, color= (255,255,255), thickness= 2)

cv2.imshow("Image",image)
#cv2.imshow("Image",rgb_image)

cv2.waitKey(0)

cv2.destroyAllWindows()

frozenset({(270, 409), (317, 402), (81, 82), (91, 181), (37, 0), (84, 17), (269, 270), (321, 375), (318, 324), (312, 311), (415, 308), (17, 314), (61, 146), (78, 95), (0, 267), (82, 13), (314, 405), (178, 87), (267, 269), (61, 185), (14, 317), (88, 178), (185, 40), (405, 321), (13, 312), (324, 308), (409, 291), (146, 91), (87, 14), (78, 191), (95, 88), (311, 310), (39, 37), (40, 39), (402, 318), (191, 80), (80, 81), (310, 415), (181, 84), (375, 291)})
x: 0.5226072669029236
y: 0.6036577224731445
z: -0.018408799543976784

(522, 402)
(543, 399)


In [12]:
# Standard imports
import cv2
import numpy as np 

# https://learnopencv.com/seamless-cloning-using-opencv-python-cpp/
# Read images
src = cv2.imread("Cropped Image.jpg")
# Define an array of endpoints of triangle
points = np.array([[14, 14],[160, 1],[158, 17],[126, 45],[95, 52],[54, 52],[22 ,28]], np.int32)
#dst = cv2.imread("clouds.jpg")
dst = cv2.imread("sorriso_teste.jpg")


#image = cv2.circle(image, (x,y), radius=0, color=(0, 0, 255), thickness=-1)

# Define an array of endpoints of triangle
points = np.array([[14, 14],[160, 1],[158, 17],[126, 45],[95, 52],[54, 52],[22 ,28]], np.int32)

#print(src.shape)
#print(dst.shape)

# Create a rough mask around the airplane.
src_mask = np.zeros(src.shape, src.dtype)
#poly = np.array([ [4,80], [30,54], [151,63], [254,37], [298,90], [272,134], [43,122] ], np.int32)
cv2.fillPoly(src_mask, [points], (255, 255, 255))

# This is where the CENTER of the airplane will be placed
center = (100,100)

source = facial_landmarks.landmark[13]
target = facial_landmarks.landmark[14]
relative_source = (int(source.x * image.shape[1]), int(source.y * image.shape[0]))
relative_target = (int(target.x * image.shape[1]), int(target.y * image.shape[0]))
print(relative_source[0])
print(relative_target)
center = (int((relative_source[0] + relative_target[0])/2),int((relative_source[1] + relative_target[1])/2)-5)
print(center)
# Clone seamlessly.
output = cv2.seamlessClone(src, dst, src_mask, center, cv2.NORMAL_CLONE)

# Save result
cv2.imwrite("fix_smile_example.jpg", output);

result = cv2.imread("fix_smile_example.jpg")
cv2.imshow("Sorriso Teste", result)

cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'facial_landmarks' is not defined

In [27]:
# Import packages
import cv2
import numpy as np

img = cv2.imread('sorriso.jpeg')
print(img.shape) # Print image shape
cv2.imshow("original", img)

# Cropping an image
cropped_image = img[384:450, 435:596]

# Display cropped image
cv2.imshow("cropped", cropped_image)

# Save the cropped image
cv2.imwrite("Cropped Image.jpg", cropped_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

(667, 1000, 3)


KeyboardInterrupt: 

In [2]:
import cv2
import numpy as np
 
# create an overlay image. You can use any image
foreground = np.ones((100,100,3),dtype='uint8')*255
# Open the camera
cap = cv2.VideoCapture(0)
# Set initial value of weights
alpha = 0.4
while True:
    # read the background
    ret, background = cap.read()
    background = cv2.flip(background,1)
    # Select the region in the background where we want to add the image and add the images using cv2.addWeighted()
    added_image = cv2.addWeighted(background[150:250,150:250,:],alpha,foreground[0:100,0:100,:],1-alpha,0)
    # Change the region with the result
    background[150:250,150:250] = added_image
    # For displaying current value of alpha(weights)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(background,'alpha:{}'.format(alpha),(10,30), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow('a',background)
    k = cv2.waitKey(10)
    # Press q to break
    if k == ord('q'):
        break
    # press a to increase alpha by 0.1
    if k == ord('a'):
        alpha +=0.1
        if alpha >=1.0:
            alpha = 1.0
    # press d to decrease alpha by 0.1
    elif k== ord('d'):
        alpha -= 0.1
        if alpha <=0.0:
            alpha = 0.0
# Release the camera and destroy all windows         
cap.release()
cv2.destroyAllWindows()

In [8]:
background = cv2.imread('sorriso.jpeg')

#background = cv2.flip(background,1)
foreground = np.ones((100,100,3),dtype='uint8')*255
added_image = cv2.addWeighted(background[150:250,150:250,:],alpha,foreground[0:100,0:100,:],1-alpha,0)
background[150:250,150:250] = added_image

cv2.imshow("original", background)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
#https://google.github.io/mediapipe/

import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
import numpy as np


foreground = np.ones((100,100,3),dtype='uint8')*255
# Set initial value of weights
alpha = 0.4
    
# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

# https://www.youtube.com/watch?v=Yg6bFRnOSbs
# https://www.youtube.com/watch?v=ObmuflX8Ank

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        src = cv2.imread("Cropped Image.jpg")
        src_mask = np.zeros(src.shape, src.dtype)

        #image = cv2.imread('sorriso.jpeg')
        rgb_image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        source = face_landmarks.landmark[13]
        target = face_landmarks.landmark[14]
        relative_source = (int(source.x * image.shape[1]), int(source.y * image.shape[0]))
        relative_target = (int(target.x * image.shape[1]), int(target.y * image.shape[0]))
        center = (int((relative_source[0] + relative_target[0])/2),int((relative_source[1] + relative_target[1])/2)-5)
        print(center)

        

        # Clone seamlessly.
        output = cv2.seamlessClone(src, image, src_mask, center, cv2.NORMAL_CLONE)
        #if face_landmarks in mp_face_mesh.FACEMESH_LIPS:
        #mp_drawing.draw_landmarks(
        #    image=image,
        #    landmark_list=face_landmarks,
        #    connections=mp_face_mesh.FACEMESH_TESSELATION,
        #    landmark_drawing_spec=None,
        #    connection_drawing_spec=mp_drawing_styles
        #    .get_default_face_mesh_tesselation_style())
        #mp_drawing.draw_landmarks(
        #    image=image,
        #    landmark_list=face_landmarks,
        #    connections=mp_face_mesh.FACEMESH_CONTOURS,
        #    landmark_drawing_spec=None,
        #    connection_drawing_spec=mp_drawing_styles
        #    .get_default_face_mesh_contours_style())
        #mp_drawing.draw_landmarks(
        #    image=image,
        #    landmark_list=face_landmarks,
        #    connections=mp_face_mesh.FACEMESH_IRISES,
        #    landmark_drawing_spec=None,
        #    connection_drawing_spec=mp_drawing_styles
        #    .get_default_face_mesh_iris_connections_style())
    # Flip the image horizontally for a selfie-view display.

    #background = cv2.flip(image, 1)
    # Select the region in the background where we want to add the image and add the images using cv2.addWeighted()
    #added_image = cv2.addWeighted(background[150:250,150:250,:],alpha,foreground[0:100,0:100,:],1-alpha,0)
    # Change the region with the result
    #background[150:250,150:250] = added_image

    cv2.imshow('MediaPipe Face Mesh', cv2.flip(output, 1))
    #cv2.imshow('MediaPipe Face Mesh', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

338
(338, 389)
(338, 383)
337
(337, 389)
(337, 383)
337
(337, 388)
(337, 383)
338
(338, 388)
(338, 383)
338
(338, 387)
(338, 382)
337
(337, 386)
(337, 381)
339
(339, 386)
(339, 381)
338
(338, 384)
(338, 379)
338
(338, 384)
(338, 379)
340
(339, 383)
(339, 377)
338
(338, 382)
(338, 377)
339
(339, 382)
(339, 377)
338
(338, 381)
(338, 376)
338
(338, 381)
(338, 376)
338
(338, 381)
(338, 376)
338
(338, 381)
(338, 376)
338
(338, 381)
(338, 376)
339
(339, 381)
(339, 376)
339
(339, 381)
(339, 376)
339
(339, 381)
(339, 376)
339
(339, 381)
(339, 376)
339
(339, 381)
(339, 376)
339
(339, 382)
(339, 377)
339
(339, 381)
(339, 376)
338
(338, 382)
(338, 376)
339
(339, 382)
(339, 377)
338
(338, 382)
(338, 376)
338
(338, 381)
(338, 376)
339
(339, 382)
(339, 377)
338
(338, 381)
(338, 376)
338
(338, 382)
(338, 377)
339
(339, 381)
(339, 376)
339
(339, 382)
(339, 377)
339
(339, 382)
(339, 377)
338
(338, 381)
(338, 376)
339
(339, 382)
(339, 376)
339
(339, 382)
(339, 377)
340
(340, 382)
(340, 376)
338
(338, 38

In [2]:
!pip install mediapipe
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached mediapipe-0.9.0.1-cp39-cp39-win_amd64.whl (49.8 MB)
  Using cached protobuf-3.20.3-cp39-cp39-win_amd64.whl (904 kB)
  Using cached opencv_contrib_python-4.7.0.68-cp37-abi3-win_amd64.whl (44.9 MB)
  Using cached flatbuffers-23.1.4-py2.py3-none-any.whl (26 kB)
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)


You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


In [10]:
import cv2
import mediapipe as mp
import numpy as np
import os

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

def facedetection():
    file = "sorriso.jpeg"

    with mp_face_detection.FaceDetection(
        model_selection=1, min_detection_confidence=0.5) as face_detection:
        image = cv2.imread(file)

        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        # Draw face detections of each face.
        if results.detections:     
            annotated_image = image.copy()
            for detection in results.detections:
                print('Nose tip:')
                print(mp_face_detection.get_key_point(
                    detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))
                mp_drawing.draw_detection(annotated_image, detection)
        cv2.imwrite('annotated_image.png', annotated_image)



# For static images:
def drawing(filename):
  IMAGE_FILES = [filename]
  drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
  with mp_face_mesh.FaceMesh(
      static_image_mode=True,
      max_num_faces=1,
      refine_landmarks=True,
      min_detection_confidence=0.5) as face_mesh:
    for idx, file in enumerate(IMAGE_FILES):
      image = cv2.imread(file)
      # Convert the BGR image to RGB before processing.
      results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

      # Print and draw face mesh landmarks on the image.
      if not results.multi_face_landmarks:
        continue
      annotated_image = image.copy()
      #print(results.multi_face_landmarks)
      for face_landmarks in results.multi_face_landmarks:
        #print('face_landmarks:', face_landmarks)
        #print(mp_face_mesh.FACEMESH_LIPS)
        #mp_drawing.draw_landmarks(
        #    image=annotated_image,
        #    landmark_list=face_landmarks,
        #    connections=mp_face_mesh.FACEMESH_TESSELATION,
        #    landmark_drawing_spec=None,
        #    connection_drawing_spec=mp_drawing_styles
        #    .get_default_face_mesh_tesselation_style())
        mp_drawing.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_LIPS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
        #mp_drawing.draw_landmarks(
        #    image=annotated_image,
        #    landmark_list=face_landmarks,
        #    connections=mp_face_mesh.FACEMESH_IRISES,
        #    landmark_drawing_spec=None,
        #    connection_drawing_spec=mp_drawing_styles
        #    .get_default_face_mesh_iris_connections_style())
      cv2.imwrite('annotated_image' + str(idx) + '.png', annotated_image)

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import os

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh      

# Read Images
image = cv2.imread("../data/bush.jpeg")
body_im = cv2.imread("../data/sorriso.jpeg")

with mp_face_mesh.FaceMesh(
      static_image_mode=True,
      max_num_faces=1,
      refine_landmarks=True,
      min_detection_confidence=0.5) as face_mesh:
  # Convert the BGR image to RGB before processing.
  results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

  
  mask_in = np.zeros(image.shape[0:2], dtype='uint8')
  points = []
  # Print and draw face mesh landmarks on the image.
  if results.multi_face_landmarks:
    
    annotated_image = image.copy()
    #print(results.multi_face_landmarks)

    face_landmarks = results.multi_face_landmarks[0]
    
    for source_idx, target_idx in mp_face_mesh.FACEMESH_LIPS:

        source = face_landmarks.landmark[source_idx]
        target = face_landmarks.landmark[target_idx]

        relative_source = (int(source.x * image.shape[1]), int(source.y * image.shape[0]))
        relative_target = (int(source.x * image.shape[1]), int(source.y * image.shape[0]))

        points.append(relative_source)
        points.append(relative_target)

array_np = np.array([[e[0], e[1]] for e in points])  

cv2.fillConvexPoly(mask_in,cv2.convexHull(array_np),color=(255,255,255))


#Mask = cv2.bitwise_not(mask_in)
#resultimage = cv2.bitwise_and(image, image, mask = Mask)

#pts1 = np.float32([[50, 50],
#                   [200, 50], 
#                   [50, 200]])
  
#pts2 = np.float32([[10, 100],
#                   [200, 50], 
#                   [100, 250]])

# Calculate the transformation matrix using cv2.getAffineTransform()
#M= cv2.getAffineTransform(pts1 , pts2)

#combined_image = image * mask_in + body_im * (1 - mask_in)


#print(f"The shape of image 1 and image 2 is {body_im.shape} and {resultimage.shape} respectively")


# The location of the center of the src in the dst
width, height, channels = body_im.shape
center = (int(height/2), int(width/2))
 
# Seamlessly clone src into dst and put the results in output
normal_clone = cv2.seamlessClone(image, body_im, mask_in, center, cv2.NORMAL_CLONE)
#mixed_clone = cv2.seamlessClone(obj, im, mask, center, cv2.MIXED_CLONE)


cv2.imshow("mask definition", normal_clone)
  
# Displaying the image
#cv2.imshow('Mask', Mask)
  
#rows, cols = body_im.shape[:2]
#warped_face = cv2.warpAffine(image,M, (cols,rows))

# waits for user to press any key
# (this is necessary to avoid Python kernel form crashing)
cv2.waitKey(0)
# closing all open windows
cv2.destroyAllWindows()

# https://www.youtube.com/watch?v=ObmuflX8Ank
# https://www.youtube.com/watch?v=Yg6bFRnOSbs


In [12]:
import cv2
import numpy as np

# Read the image
img = cv2.imread('../data/bush.jpeg')
rows, cols = img.shape[:2]

# Define the 3 pairs of corresponding points 
pts1 = np.float32([[50, 50],
                   [200, 50], 
                   [50, 200]])
  
pts2 = np.float32([[10, 100],
                   [200, 50], 
                   [100, 250]])

# Calculate the transformation matrix using cv2.getAffineTransform()
M= cv2.getAffineTransform(pts1 , pts2)
print(img.shape)
print(pts2)
print(M)

# Apply the affine transformation using cv2.warpAffine()
dst = cv2.warpAffine(img, M, (cols,rows))

# Display the image
out = cv2.hconcat([img, dst])
cv2.imshow('Output', out)
cv2.waitKey(0)

(847, 640, 3)
[[ 10. 100.]
 [200.  50.]
 [100. 250.]]
[[  1.26666667   0.6        -83.33333333]
 [ -0.33333333   1.          66.66666667]]


-1

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh      

# Leitura entrada
image_in = cv2.imread("../data/bush.jpeg")

# Leitura Saida
image_out = cv2.imread("../data/sorriso.jpeg")

with mp_face_mesh.FaceMesh(
      static_image_mode=True,
      max_num_faces=1,
      refine_landmarks=True,
      min_detection_confidence=0.5) as face_mesh:
  # Convert the BGR image to RGB before processing.
  results = face_mesh.process(cv2.cvtColor(image_in, cv2.COLOR_BGR2RGB))

  # Mascara para sorriso
  mask_in = np.zeros(image_in.shape[0:2], dtype='uint8')
  points = []
  if results.multi_face_landmarks:

    annotated_image = image_in.copy()
    default_image = image_in.copy()

    # landmarks do primeiro rosto
    face_landmarks = results.multi_face_landmarks[0]
    #frozen = frozenset({(270, 409), (317, 402), (81, 82), (91, 181), (37, 0), (84, 17), (269, 270), (321, 375), (318, 324), 
    # (312, 311), (415, 308), (17, 314), (61, 146), (78, 95), (0, 267), (82, 13), (314, 405), (178, 87), (267, 269), (61, 185), 
    # (14, 317), (88, 178), (185, 40), (405, 321), (13, 312), (324, 308), (409, 291), (146, 91), (87, 14), (78, 191), (95, 88), (311, 310), (39, 37), (40, 39), (402, 318), (191, 80),
    #  (80, 81), (310, 415), (181, 84), (375, 291)})
    frozen = frozenset({ (317, 402),(81, 82), (318, 324), (312, 311),(415, 308), (78, 95),(82, 13), (178, 87),
     (14, 317), (88, 178), (13, 312), (324, 308),
    (87, 14),(78, 191),(95, 88),(311, 310),(402, 318),(191, 80),(80, 81),(310, 415)})
    

    #for source_idx, target_idx in mp_face_mesh.FACEMESH_LIPS:
    color = (255, 0, 0)
    radius = 2
    # Line thickness of 2 px
    thickness = 2
    # font
    font = cv2.FONT_HERSHEY_SIMPLEX
      
      # org
    org = (00, 185)
      
      # fontScale
    fontScale = 1
      
      # Red color in BGR
    color = (0, 0, 255)
      
      # Line thickness of 2 px
    thickness = 2
    for source_idx, target_idx in frozen:
      


      #print(mp_face_mesh.FACEMESH_LIPS)
      mp_drawing.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks,
            connections=frozen,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())

      #mp_drawing.draw_landmarks(
      #      image=default_image,
      #      landmark_list=face_landmarks,
      #      connections=mp_face_mesh.FACEMESH_LIPS,
      #      landmark_drawing_spec=None,
      #      connection_drawing_spec=mp_drawing_styles
      #      .get_default_face_mesh_contours_style())

      source = face_landmarks.landmark[source_idx]
      target = face_landmarks.landmark[target_idx]

      relative_source = (int(source.x * image_in.shape[1]), int(source.y * image_in.shape[0]))
      relative_target = (int(target.x * image_in.shape[1]), int(target.y * image_in.shape[0]))
      print("source idx {}".format(source_idx))
      print("target_idx idx {}".format(target_idx))
      #image_in = cv2.putText(image_in, str(source_idx), relative_source, font, fontScale, 
      #           color, thickness, cv2.LINE_AA, False)
      if source_idx == 87 or source_idx == 82:
      #if source_idx == 82:
            image_in = cv2.circle(image_in, relative_target, radius, color, thickness)
      #image_in = cv2.circle(image_in, relative_source, radius, color, thickness)

      points.append(relative_source)
      points.append(relative_target)

array_np = np.array([[e[0], e[1]] for e in points])  

file = open("sample.txt", "w+")
# Saving the array in a text file
content = str(array_np)
file.write(content)
file.close()

cv2.fillConvexPoly(mask_in,cv2.convexHull(array_np),color=(255,255,255))

with mp_face_mesh.FaceMesh(
      static_image_mode=True,
      max_num_faces=1,
      refine_landmarks=True,
      min_detection_confidence=0.5) as face_mesh:
  # Convert the BGR image to RGB before processing.
  results = face_mesh.process(cv2.cvtColor(image_out, cv2.COLOR_BGR2RGB))
  face_landmarks = results.multi_face_landmarks[0]
  source = face_landmarks.landmark[82]
  target = face_landmarks.landmark[87]

  relative_source = (int(source.x * image_out.shape[1]), int(source.y * image_out.shape[0]))
  relative_target = (int(target.x * image_out.shape[1]), int(target.y * image_out.shape[0]))

# The location of the center of the src in the dst
width, height, channels = image_out.shape
print(image_out.shape)
print(relative_source[0])
print(relative_target[0])
print(int(relative_source[0] +  ((relative_target[0] -  relative_source[0])/2)))
y_center = int(relative_source[1] +  ((relative_target[1] -  relative_source[1])/2))
#center = (int(relative_source[1] + (relative_source[1] - relative_target[1]) /2), int(width/2))
center = (relative_source[0], y_center)
 
# Seamlessly clone src into dst and put the results in output
normal_clone = cv2.seamlessClone(image_in, image_out, mask_in, center, cv2.NORMAL_CLONE)
mixed_clone = cv2.seamlessClone(image_in, image_out, mask_in, center, cv2.MIXED_CLONE)
#mixed_clone = cv2.seamlessClone(obj, im, mask, center, cv2.MIXED_CLONE)


#cv2.imshow("normal_clone definition", normal_clone)
cv2.imshow("default_image definition", image_in)
cv2.imshow("annotated_image definition", normal_clone)

  
cv2.waitKey(0)
# closing all open windows
cv2.destroyAllWindows()

# https://www.youtube.com/watch?v=ObmuflX8Ank
# https://www.youtube.com/watch?v=Yg6bFRnOSbs
# https://www.youtube.com/watch?v=ExToU4KzDm8
# https://github.com/numfocus/YouTubeVideoTimestamps
# https://github.com/sylwekb/face_swap


source idx 317
target_idx idx 402
source idx 81
target_idx idx 82
source idx 13
target_idx idx 312
source idx 318
target_idx idx 324
source idx 324
target_idx idx 308
source idx 312
target_idx idx 311
source idx 415
target_idx idx 308
source idx 78
target_idx idx 95
source idx 87
target_idx idx 14
source idx 78
target_idx idx 191
source idx 82
target_idx idx 13
source idx 95
target_idx idx 88
source idx 311
target_idx idx 310
source idx 402
target_idx idx 318
source idx 191
target_idx idx 80
source idx 80
target_idx idx 81
source idx 178
target_idx idx 87
source idx 310
target_idx idx 415
source idx 14
target_idx idx 317
source idx 88
target_idx idx 178
(667, 1000, 3)
496
501
498


In [1]:
import cv2
 
img = cv2.imread('../data/sorriso.jpeg', cv2.IMREAD_UNCHANGED)
img_out = cv2.imread('../data/bush.jpeg', cv2.IMREAD_UNCHANGED)
 
print('Original Dimensions : ',img.shape)
print('Original img_out Dimensions : ',img_out.shape)
 
scale_percent = 60 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (img_out.shape[1], img_out.shape[0])
  
# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
 
print('Resized Dimensions : ',resized.shape)
print(img_out.shape[:2])
 
#cv2.imshow("Resized image", resized)
cv2.imshow("Out image", img_out)
cv2.waitKey(0)
cv2.destroyAllWindows()

Original Dimensions :  (667, 1000, 3)
Original img_out Dimensions :  (847, 640, 3)
Resized Dimensions :  (847, 640, 3)
(847, 640)
